# Introduction to LangChain.js

Welcome! This short course will introduce you to [LangChain.js](https://github.com/langchain-ai/langchainjs), a framework for building large language model (LLM) powered, context-aware, reasoning applications. By the end of the course, you'll learn all the concepts you need to create your own version of a popular example of this type of app: a "chat with data" chain that lets you ask questions about a document's contents using natural language.

This course follows on from the previous DeepLearning.ai courses on the Python version of LangChain. Much like LangChain.js itself, it's intended for web developers and others in the broader JavaScript ecosystem interested in building with LLMs, and will dive more deeply into features like streaming and integration with standard web/JavaScript APIs.

**Note:** This notebook uses the [Deno](https://deno.com/) Jupyter kernel, and has slightly different import statements from those you may be familiar with in Node and web runtimes. You can convert them to Node/web imports by swapping `npm:langchain@0.0.178` for `langchain`. To run these course notebooks locally, see [these setup instructions](https://docs.deno.com/runtime/manual/tools/jupyter).

**Note:** Throughout this course, we'll link to explorable traces in [LangSmith](https://smith.langchain.com/), an LLM-focused observability platform by the company behind the open source framework, that illustrate how the different example chains work. If you're following along locally, you'll need to set a few environment variables [as documented here](https://docs.smith.langchain.com/).

## Building Blocks: LLMs

Let's start with one of the most fundamental pieces of LangChain: the language model. LangChain includes two different types of language models: 

1. LLMs, which take a string as input and returns a string. An example of this is OpenAI's `text-davinci-003`, also known as GPT-3.
2. Chat Models, which take a list of messages as input and return a message. An example of this is OpenAI's `gpt-4`.

As strings, LLMs inputs and outputs are easy to visualize, so let's look at what calling a chat model directly looks like:

In [1]:
// Deno.env.set("OPENAI_API_KEY", "");
// Deno.env.set("LANGCHAIN_TRACING_V2", "true");
// Deno.env.set("LANGCHAIN_SESSION", "");
// Deno.env.set("LANGCHAIN_API_KEY", "");

import { ChatOpenAI } from "npm:langchain@0.0.178/chat_models/openai";
import { HumanMessage } from "npm:langchain@0.0.178/schema";

const model = new ChatOpenAI({});

await model.invoke([
  new HumanMessage("Tell me a joke."),
]);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Why don't scientists trust atoms? \n\nBecause they make up everything!",
    additional_kwargs: { function_call: undefined }
  },
  lc_namespace: [ "langchain", "schema" ],
  content: "Why don't scientists trust atoms? \n\nBecause they make up everything!",
  name: undefined,
  additional_kwargs: { function_call: undefined }
}

Above, we pass in an array with a single `HumanMessage` as input, and receive a single `AIMessage` as output. Messages have a `content` field containing the text value of the message, and an associated `role` that corresponds to the entity sending the message. 

**Note:** While this course will primarily use OpenAI's `gpt-3.5-turbo` chat model, LangChain supports models from many different providers, and you can try swapping the provided class in any of the code examples.

## Building Blocks: Prompt Templates

While calling models in isolation can be useful, it is often more convenient to factor out the logic behind model inputs into a reusable, parameterized component. For this purpose, LangChain includes prompt templates, which are responsible for formatting user input for later model calls. Input variables are denoted by curly braces within the template, and will be substituted into the final formatted value. 

Prompt templates are also useful for smoothing over some of the differences in model input types. Below, we construct a prompt template directly from a string:

In [2]:
import { ChatPromptTemplate } from "npm:langchain@0.0.178/prompts";

const prompt = ChatPromptTemplate.fromTemplate(`What are three good names for a company that makes {product}?`);

But we can use this prompt template to generate both string input for an LLM:

In [3]:
await prompt.format({
  product: "colorful socks",
});

"Human: What are three good names for a company that makes colorful socks?"

Or a message array for chat models:

In [4]:
await prompt.formatMessages({
  product: "colorful socks",
});

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes colorful socks?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain", "schema" ],
    content: "What are three good names for a company that makes colorful socks?",
    name: undefined,
    additional_kwargs: {}
  }
]

Though we can pass these formatted values directly into a model, there's a more elegant way to use prompts and models together that we'll go over next.

## Building Blocks: LangChain Expression Language

LangChain Expression Language (LCEL) is a composable syntax for chaining LangChain modules together. Objects that are compatible with LCEL are called `Runnables`.

We can construct a simple chain from the prompt and model we declared above like this:

In [5]:
const chain = prompt.pipe(model);

The input of the chain is the same as the first step in the sequence, which in this case is an object with a single `product` property. The prompt template is invoked with this input, then passes the properly formatted result as input into the next step of the chain, the chat model. Here's what it looks like in action:

In [6]:
await chain.invoke({
  product: "colorful socks",
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "1. ChromaSockz\n2. RainbowThreads\n3. VibrantFeet",
    additional_kwargs: { function_call: undefined }
  },
  lc_namespace: [ "langchain", "schema" ],
  content: "1. ChromaSockz\n2. RainbowThreads\n3. VibrantFeet",
  name: undefined,
  additional_kwargs: { function_call: undefined }
}

We can see that the result is an `AIMessage` with three good names for a company that makes colorful socks.

## Building Blocks: Output Parser

The final consideration we'll go over in this section is formatting our output. For example, it is often easier to work with the raw string value of a chat model's output rather than an AI message. The LangChain abstraction for this is called an output parser. 

Below, we redefine our chain with an output parser that coerces the message output from the chat model into a string as a third and final step:

In [7]:
import { StringOutputParser } from "npm:langchain@0.0.178/schema/output_parser";

const outputParser = new StringOutputParser();

const nameGenerationChain = prompt.pipe(model).pipe(outputParser);

And now, if we invoke the chain, we can see that the output is a raw string:

In [8]:
await nameGenerationChain.invoke({
  product: "fancy cookies",
});

"1. Dolce Delights\n2. Gourmet Crumbles\n3. Elegance Bites"

You can see a LangSmith trace of the above example [here](https://smith.langchain.com/public/02c0469b-f3b4-4681-b006-da74df897dfa/r).

These three pieces form the core of many more complicated chains.

## Streaming

One of the many advantages to LCEL is that chains composed in this fashion get certain methods automatically. One useful one is `.stream()`, which returns output from the chain in an iterable stream. Because LLM responses often take a long time to finish, streaming is useful in situations where showing feedback quickly is important.

Here's an example with the chain we just composed:

In [9]:
const stream = await nameGenerationChain.stream({
  product: "really cool robots",
});

for await (const chunk of stream) {
  console.log(chunk);
}


1
.
 Rob
o
Tech
 Innov
ations


2
.
 Cyber
tron
 Robotics


3
.
 Fut
u
Robot
ics



Above, the `model` emits partial message chunks, and the output parser transforms those streamed chunks from the model as they are generated, resulting in immediate string output.

This stream is a special `ReadableStream` that implements the iterator interface and can be passed directly to e.g. a `fetch` built-in `Response` object and returned in popular web frameworks like Next.js.

In [10]:
const stream = await nameGenerationChain.stream({
  product: "superb owls",
});

new Response(stream);

Response {
  body: IterableReadableStream { locked: false },
  bodyUsed: false,
  headers: Headers {},
  ok: true,
  redirected: false,
  status: 200,
  statusText: "",
  url: ""
}

## Chaining with `RunnableMap`

While sequentially chaining calls together like this can be useful on its own, more complex chains often need to combine inputs from different sources and steps. For example, let's say we want to take the company name outputs generated by the above chain and pass it as one input of several into another chain that picks the best one. One way to do this is to add a `RunnableMap` to the sequence. We also declare the sequence using the alternate `RunnableSequence.from()` method for readability:

In [11]:
import { RunnableSequence, RunnableMap } from "npm:langchain@0.0.178/schema/runnable";

const finalPrompt = ChatPromptTemplate.fromTemplate(`Pick the best name from {names} based on {final_criteria} and explain why.`);

const combinedChain = RunnableSequence.from([
  RunnableMap.from({
    names: nameGenerationChain,
    final_criteria: (input) => input.final_criteria,
  }),
  finalPrompt,
  model,
  new StringOutputParser(),
]);

Each property in the `RunnableMap` gets called in parallel with the input from the previous step. In the above example, since the `RunnableMap` is the first step, it is called with the input to the overall chain. The output of the `RunnableMap` is an object with the result of that call. Since we want to pass `final_criteria` through to `finalPrompt`, we simply extract the property from the input.

We can see the result below:

In [12]:
await combinedChain.invoke({
  product: "wooden cars",
  final_criteria: "sustainability",
});

'The best name among the three options based on sustainability would be "EcoCarve Automotive." \n' +
  "\n" +
  "This"... 496 more characters

This [LangSmith trace](https://smith.langchain.com/public/ed4d767a-49d5-427c-97fe-fc2dc3f5aba5/r) shows what's going on under the hood - the `nameGenerationChain` we defined above generated three good names for a company that makes wooden cars, and the output was passed to our `finalPrompt` along with the `final_criteria` we originally passed to make a final judgement of the best name.